## 🧮 Evaluation Metrics — Top-k, Perplexity, BLEU, ROUGE

### **1. Top-k Accuracy**
**What:** Measures how often the correct class is among the model’s top-k predictions.  
**Formula:**  
$$
\text{Top-k Accuracy} = \frac{1}{N}\sum_{i=1}^{N} \mathbf{1}(y_i \in \text{Top-k}(\hat{y}_i))
$$  
**Where used:**  
- Image classification, recommendation systems, retrieval tasks.  
- Top-1 = normal accuracy; higher k captures model uncertainty.  

**Limitations:**  
- Ignores ranking within top-k (e.g., being 2nd or 5th counts the same).  
- Not suitable for regression or sequence generation.

In [25]:
from collections import Counter
from math import exp, log
from typing import List, Iterable, Tuple


def top_k_accuracy(y_true: List[int], y_pred_scores: List[Iterable[float]], k: int = 5) -> float:
    """
    Top-k accuracy: fraction of examples where the true class index is among the top-k predicted scores.
    - y_true: list of true class indices (len = N)
    - y_pred_scores: list of score vectors (logits or probs) per example (shape N x C)
    - k: top-k threshold
    """
    correct = 0

    for t, scores in zip(y_true, y_pred_scores):
        topk = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:k]
        correct += int(t in topk)

    return correct / len(y_true) if y_true else 0.0

### **2. Perplexity**
**What:** Measures how “surprised” a language model is by the data. Lower = better.  
**Formula:**  
$$
\text{PPL} = e^{-\frac{1}{T} \sum_{t=1}^{T} \log p(w_t | w_{<t})}
$$  
**Where used:**  
- Evaluating language models (GPT, RNN, etc.) on next-token prediction tasks.  

**Limitations:**  
- Only works when you can compute exact probabilities.  
- Doesn’t always correlate with human text quality.  
- Sensitive to tokenization.

In [26]:
def perplexity_from_nll(nll_tokens: List[float]) -> float:
    """
    Perplexity from per-token negative log-likelihoods (natural log).
    PPL = exp( mean(NLL) )
    - nll_tokens: list of -log p(w_t | context) for each token (base e)
    """
    if not nll_tokens:
        return float('nan')
    return exp(sum(nll_tokens) / len(nll_tokens))


def perplexity_from_logprobs(logprobs: List[float]) -> float:
    """
    Perplexity when given per-token log-probabilities (natural log).
    - logprobs: list of log p(w_t | context)
    """
    if not logprobs:
        return float('nan')
    mean_nll = -sum(logprobs) / len(logprobs)
    return exp(mean_nll)

In [27]:
y_true = [2, 0]
y_pred = [[0.1, 0.2, 0.7], [0.6, 0.3, 0.1]]
print("Top-1 acc:", top_k_accuracy(y_true, y_pred, k=1))  # 1.0
print("Top-2 acc:", top_k_accuracy(y_true, y_pred, k=2))  # 1.0
print("PPL:", perplexity_from_logprobs([-0.2, -0.1, -0.3]))

Top-1 acc: 1.0
Top-2 acc: 1.0
PPL: 1.2214027581601699



### **3. BLEU (Bilingual Evaluation Understudy)**
**What:** Precision-based metric comparing n-gram overlap between candidate and reference text.  
**Formula:**  
$$
\text{BLEU} = \text{BP} \cdot \exp\left( \sum_{n=1}^{N} w_n \log p_n \right)
$$  
where  
$$
p_n = \frac{\text{\# matched n-grams}}{\text{\# candidate n-grams}}, \quad
\text{BP} =
\begin{cases}
1, & c > r \\
e^{1 - \frac{r}{c}}, & c \le r
\end{cases}
$$  
**Where used:**  
- Machine translation, summarization, text generation.

**Limitations:**  
- Ignores recall (only precision).  
- Penalizes valid paraphrases (exact n-gram match only).  
- Poor at sentence-level evaluation; better averaged over corpus.

In [28]:
# -----------------------------
# BLEU (sentence-level, single candidate, multi-reference)
# -----------------------------

def _ngrams(tokens: List[str], n: int) -> Counter:
    return Counter(tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)) if n > 0 else Counter()

def _brevity_penalty(c_len: int, r_len: int) -> float:
    if c_len == 0:
        return 0.0
    return 1.0 if c_len > r_len else exp(1 - r_len / c_len)

def bleu(candidate: List[str], references: List[List[str]], max_n: int = 4, smooth: bool = True) -> float:
    """
    BLEU (Papineni et al.): geometric mean of clipped n-gram precisions * brevity penalty.
    - candidate: tokenized hypothesis
    - references: list of tokenized references
    - max_n: up to n-gram order (default 4)
    - smooth: add-1 smoothing for zero counts (Chen & Cherry style)
    """
    if not candidate:
        return 0.0

    # choose reference length closest to candidate (ties -> shortest)
    c_len = len(candidate)
    r_lens = [len(r) for r in references]
    r_len = min(r_lens, key=lambda rl: (abs(rl - c_len), rl))

    precisions = []
    for n in range(1, max_n + 1):
        c_grams = _ngrams(candidate, n)
        max_ref = Counter()
        for r in references:
            max_ref |= _ngrams(r, n)  # elementwise max for clipped counts
        overlap = sum((c_grams & max_ref).values())
        total = max(1, sum(c_grams.values()))
        if overlap == 0 and smooth:
            # add-1 smoothing
            p_n = (overlap + 1) / (total + 1)
        else:
            p_n = overlap / total
        precisions.append(p_n)

    # geometric mean in log space
    # If any precision is zero and no smoothing, BLEU becomes 0 (handled naturally).
    log_prec = sum((0 if p == 0 else log(p)) for p in precisions) / max_n
    bp = _brevity_penalty(c_len, r_len)
    return bp * exp(log_prec)

In [29]:
# BLEU
cand = "the cat is on the mat".split()
refs = ["there is a cat on the mat".split(), "a cat is on the mat".split()]
print("BLEU:", bleu(cand, refs, max_n=4, smooth=True))

BLEU: 0.7598356856515925


### **4. ROUGE (Recall-Oriented Understudy for Gisting Evaluation)**
**What:** Recall-based metric measuring overlap between candidate and reference summaries.  
- **ROUGE-N:** n-gram overlap (recall, precision, F1).  
  $$
  \text{Recall} = \frac{\text{overlap}}{\text{ref total}}, \quad
  \text{Precision} = \frac{\text{overlap}}{\text{cand total}}, \quad
  F1 = \frac{2PR}{P+R}
  $$
- **ROUGE-L:** Based on Longest Common Subsequence (LCS).  

**Where used:**  
- Summarization, text generation, dialogue response quality.  

**Limitations:**  
- Overly lexical (word-form sensitive).  
- Doesn’t capture meaning or fluency.  
- High recall may come from redundant text.

In [30]:
# -----------------------------
# ROUGE-1 / ROUGE-2 (F1, Precision, Recall) and ROUGE-L (F1)
# -----------------------------
def _overlap_counts(cand: List[str], ref: List[str], n: int) -> Tuple[int, int, int]:
    """Return (overlap, cand_total, ref_total) for ROUGE-n using multiset overlap."""
    c_grams = _ngrams(cand, n)
    r_grams = _ngrams(ref, n)
    overlap = sum((c_grams & r_grams).values())
    return overlap, sum(c_grams.values()), sum(r_grams.values())

def _prf1(overlap: int, cand_total: int, ref_total: int) -> Tuple[float, float, float]:
    p = overlap / cand_total if cand_total else 0.0
    r = overlap / ref_total if ref_total else 0.0
    f1 = 2*p*r / (p + r) if (p + r) > 0 else 0.0
    return p, r, f1

def rouge_n(candidate: List[str], reference: List[str], n: int = 1) -> Tuple[float, float, float]:
    """
    ROUGE-n (n=1 or 2 typical): multiset n-gram overlap.
    Returns (precision, recall, F1).
    """
    overlap, c_tot, r_tot = _overlap_counts(candidate, reference, n)
    return _prf1(overlap, c_tot, r_tot)

def _lcs_len(a: List[str], b: List[str]) -> int:
    # O(|a|*|b|) ie O(n2) but won't be issue for this scale
    m, n = len(a), len(b)
    dp = [0]*(n+1)
    for i in range(1, m+1):
        prev = 0
        for j in range(1, n+1):
            temp = dp[j]
            if a[i-1] == b[j-1]:
                dp[j] = prev + 1
            else:
                dp[j] = max(dp[j], dp[j-1])
            prev = temp
    return dp[-1]

def rouge_l(candidate: List[str], reference: List[str]) -> Tuple[float, float, float]:
    """
    ROUGE-L: based on Longest Common Subsequence.
    Returns (precision, recall, F1) where overlap = LCS length.
    """
    lcs = _lcs_len(candidate, reference)
    p = lcs / len(candidate) if candidate else 0.0
    r = lcs / len(reference) if reference else 0.0
    f1 = 2*p*r / (p + r) if (p + r) > 0 else 0.0
    return p, r, f1

In [31]:
# ROUGE
ref = "a cat sat on the mat".split()
print("ROUGE-1 (P,R,F1):", rouge_n(cand, ref, n=1))
print("ROUGE-2 (P,R,F1):", rouge_n(cand, ref, n=2))
print("ROUGE-L (P,R,F1):", rouge_l(cand, ref))

ROUGE-1 (P,R,F1): (0.6666666666666666, 0.6666666666666666, 0.6666666666666666)
ROUGE-2 (P,R,F1): (0.4, 0.4, 0.4000000000000001)
ROUGE-L (P,R,F1): (0.6666666666666666, 0.6666666666666666, 0.6666666666666666)


### ✅ Quick Mental Model
| Metric | Domain | Focus | High Score Means | Weakness |
|:--|:--|:--|:--|:--|
| **Top-k Accuracy** | Classification | Rank of true label | Model often includes truth | Ignores rank order |
| **Perplexity** | Language modeling | Predictive confidence | Model predicts next word well | Not aligned with human quality |
| **BLEU** | Translation | n-gram precision | Candidate close to reference | Misses paraphrases |
| **ROUGE** | Summarization | n-gram recall / LCS | Captures key content | Surface-form only |

---
# Cross-Validation (K-Fold) with Dummy Dataset

In this notebook, we explore **K-Fold Cross-Validation**, a robust technique for evaluating model performance by repeatedly training and testing on different partitions of the dataset. Instead of relying on a single train/test split, K-Fold validation divides the data into *K equal folds* and iteratively uses each fold as the test set while the remaining folds serve as the training set.

**Why it matters:**
- Reduces variance in evaluation results.
- Utilizes all data for both training and validation.
- Provides a better estimate of generalization performance.

We'll:
1. Generate a dummy dataset using `make_classification`.
2. Implement K-Fold cross-validation using `sklearn.model_selection.KFold`.
3. Train a simple model (e.g., Logistic Regression).
4. Compute and interpret average accuracy across folds.


In [32]:
# K-Fold Cross Validation with Dummy Dataset

from sklearn.datasets import make_classification
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
import numpy as np

# Generate a dummy dataset
X, y = make_classification(
    n_samples=200, n_features=5, n_informative=3,
    n_redundant=0, n_classes=2, random_state=42
)

# Define the model
model = LogisticRegression(max_iter=1000)

# Define K-Fold Cross-Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Evaluate model using cross-validation
scores = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')

print("Fold Accuracies:", scores)
print(f"Mean Accuracy: {scores.mean():.4f} ± {scores.std():.4f}")

Fold Accuracies: [0.925 0.975 0.875 0.875 0.925]
Mean Accuracy: 0.9150 ± 0.0374


In [37]:
# K-Fold setup
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []

# Loop over folds
for fold, (train_idx, val_idx) in enumerate(kfold.split(X_tensor)):
    X_train, X_val = X_tensor[train_idx], X_tensor[val_idx]
    y_train, y_val = y_tensor[train_idx], y_tensor[val_idx]

    # New model for each fold
    model = MLP()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    train_model(model, optimizer, criterion, X_train, y_train)
    acc = evaluate_model(model, X_val, y_val)
    fold_accuracies.append(acc)

    print(f"Fold {fold + 1} Accuracy: {acc:.4f}")

# Report overall performance
print(f"\nMean Accuracy: {np.mean(fold_accuracies):.4f} ± {np.std(fold_accuracies):.4f}")

Fold 1 Accuracy: 0.9500
Fold 2 Accuracy: 1.0000
Fold 3 Accuracy: 0.8750
Fold 4 Accuracy: 0.9000
Fold 5 Accuracy: 0.9000

Mean Accuracy: 0.9250 ± 0.0447


# Pytorch Version: Manual K-Fold Cross-Validation

In this section, we manually implement **K-Fold Cross-Validation** using PyTorch to understand the full training and evaluation workflow.

**Goal:**  
Perform repeated training and validation across K folds and average the results to obtain a more reliable estimate of the model’s performance.

**Steps:**
1. Create a dummy dataset using `sklearn.datasets.make_classification`.
2. Convert it into PyTorch tensors.
3. Split the dataset using `KFold`.
4. Train a simple MLP model on each fold.
5. Evaluate and report mean accuracy across folds.


In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import numpy as np

In [34]:
# Generate dummy dataset
X, y = make_classification(
    n_samples=200, n_features=5, n_informative=3,
    n_redundant=0, n_classes=2, random_state=42
)

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

In [35]:
# Define a simple MLP model
class MLP(nn.Module):
    def __init__(self, input_dim=5, hidden_dim=16, output_dim=2):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.net(x)

In [36]:
# Define training function
def train_model(model, optimizer, criterion, X_train, y_train, epochs=50):
    model.train()
    for _ in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

# Define evaluation function
def evaluate_model(model, X_val, y_val):
    model.eval()
    with torch.no_grad():
        outputs = model(X_val)
        preds = torch.argmax(outputs, dim=1)
        acc = (preds == y_val).float().mean().item()
    return acc